In [9]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [10]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch_prep_policy.gdb", 'policy_override.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])


In [11]:
# read in data
polygons = r'.\2019\policy_override_2019.gdb\policy_override_polygons_2019'
polygons_df = pd.DataFrame.spatial.from_featureclass(polygons)
polygons_df.head()

,OBJECTID,id,max_dua,max_dua_new,max_far,max_far_new,year,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,locnote,mponote,AreaName,AreaType,phase_begin_year,phase_end_year,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,SHAPE
0,1,1,10.0,50.0,0.8,1.5,2019.0,1,1,0,1,1,0,0,0,11011000,WFRCCenters,WFRC,West Layton Town Center,City Center,2031,2040,1,,,,,"{""rings"": [[[415395.8512000004, 4546928.454], ..."
1,2,2,0.4,NaN,0.6,NaN,2019.0,0,1,1,1,1,0,0,0,01111000,WFRCCenters,WFRC,Layton Industrial Center,Industrial District,2031,2040,1,,,,,"{""rings"": [[[416374.0889999997, 4547760.0077],..."
2,3,3,30.0,100.0,2.0,4.0,2019.0,0,1,1,1,1,0,0,0,01111000,WFRCCenters,WFRC,Layton Midtown,Urban Center,2019,2030,1,,,,,"{""rings"": [[[417032.33100000024, 4549527.56279..."
3,4,4,30.0,100.0,1.9,4.0,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Layton Downtown,Urban Center,2031,2040,1,,,,,"{""rings"": [[[419554.49820000026, 4545424.31719..."
4,5,5,10.0,50.0,0.8,1.5,2037.0,1,1,0,1,1,0,0,0,11011000,WFRCCenters,WFRC,Gentile & Fairfield Town Center,City Center,2041,2050,1,,,,,"{""rings"": [[[420429.2611999996, 4546442.0261],..."


In [12]:
phased = polygons_df[polygons_df['phase_count'] > 1].copy()
phased

,OBJECTID,id,max_dua,max_dua_new,max_far,max_far_new,year,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,locnote,mponote,AreaName,AreaType,phase_begin_year,phase_end_year,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,SHAPE
148,149,149,65.0,NaN,0.0,NaN,2019.0,1,1,0,1,1,0,0,0,11011000,CBDStaging,WFRC,Ogden Downtown,Metropolitan Center,2019,2030,3,"2019,2025,2035","65,95,125","0,0,0","11011000,11011000,11011000","{""rings"": [[[417671.25289999973, 4562349.66400..."
211,212,212,65.0,NaN,0.0,NaN,2019.0,1,1,0,1,1,0,0,0,11011000,CBDStaging,WFRC,Salt Lake City CBD,Metropolitan Center,2019,2030,3,"2019,2025,2035","65,95,125","0,0,0","11011000,11011000,11011000","{""rings"": [[[424545.88900000043, 4511377.4616]..."


In [13]:
df = pd.DataFrame()
for index, row in phased.iterrows():
    phase_years = row['phase_years'].split(',')
    phase_max_duas = row['phase_maxduas'].split(',')
    phase_max_fars = row['phase_maxfars'].split(',')
    phase_types = row['phase_types'].split(',')

    # check to make sure phase lists are equal
    if len(set([len(phase_years), len(phase_max_duas), len(phase_max_fars), len(phase_types)])) != 1:
        raise ValueError("Missmatch in phase length values for id: {}".format(row['id']))

    
    for i in range(len(phase_years)):
        new_row = row
        new_row['year'] =  phase_years[i]
        new_row['max_dua'] =  phase_max_duas[i]
        new_row['max_far'] =  phase_max_fars[i]
        new_row['type1'] =  phase_types[i][0]
        new_row['type2'] =  phase_types[i][1]
        new_row['type3'] =  phase_types[i][2]
        new_row['type4'] =  phase_types[i][3]
        new_row['type5'] =  phase_types[i][4]
        new_row['type6'] =  phase_types[i][5]
        new_row['type7'] =  phase_types[i][6]
        new_row['type8'] =  phase_types[i][7]
        df = df.append(new_row)
        

In [14]:
df

,AreaName,AreaType,OBJECTID,SHAPE,id,locnote,max_dua,max_dua_new,max_far,max_far_new,mponote,phase_begin_year,phase_count,phase_end_year,phase_maxduas,phase_maxfars,phase_types,phase_years,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,year
148,Ogden Downtown,Metropolitan Center,149.0,"{'rings': [[[417671.25289999973, 4562349.66400...",149.0,CBDStaging,65,NaN,0,NaN,WFRC,2019.0,3.0,2030.0,"65,95,125","0,0,0","11011000,11011000,11011000","2019,2025,2035",1,1,0,1,1,0,0,0,11011000,2019
148,Ogden Downtown,Metropolitan Center,149.0,"{'rings': [[[417671.25289999973, 4562349.66400...",149.0,CBDStaging,95,NaN,0,NaN,WFRC,2019.0,3.0,2030.0,"65,95,125","0,0,0","11011000,11011000,11011000","2019,2025,2035",1,1,0,1,1,0,0,0,11011000,2025
148,Ogden Downtown,Metropolitan Center,149.0,"{'rings': [[[417671.25289999973, 4562349.66400...",149.0,CBDStaging,125,NaN,0,NaN,WFRC,2019.0,3.0,2030.0,"65,95,125","0,0,0","11011000,11011000,11011000","2019,2025,2035",1,1,0,1,1,0,0,0,11011000,2035
211,Salt Lake City CBD,Metropolitan Center,212.0,"{'rings': [[[424545.88900000043, 4511377.4616]...",212.0,CBDStaging,65,NaN,0,NaN,WFRC,2019.0,3.0,2030.0,"65,95,125","0,0,0","11011000,11011000,11011000","2019,2025,2035",1,1,0,1,1,0,0,0,11011000,2019
211,Salt Lake City CBD,Metropolitan Center,212.0,"{'rings': [[[424545.88900000043, 4511377.4616]...",212.0,CBDStaging,95,NaN,0,NaN,WFRC,2019.0,3.0,2030.0,"65,95,125","0,0,0","11011000,11011000,11011000","2019,2025,2035",1,1,0,1,1,0,0,0,11011000,2025
211,Salt Lake City CBD,Metropolitan Center,212.0,"{'rings': [[[424545.88900000043, 4511377.4616]...",212.0,CBDStaging,125,NaN,0,NaN,WFRC,2019.0,3.0,2030.0,"65,95,125","0,0,0","11011000,11011000,11011000","2019,2025,2035",1,1,0,1,1,0,0,0,11011000,2035


In [15]:
non_phased = polygons_df[polygons_df['phase_count'] == 1].copy()
non_phased

,OBJECTID,id,max_dua,max_dua_new,max_far,max_far_new,year,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,locnote,mponote,AreaName,AreaType,phase_begin_year,phase_end_year,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,SHAPE
0,1,1,10.0,50.0,0.8,1.5,2019.0,1,1,0,1,1,0,0,0,11011000,WFRCCenters,WFRC,West Layton Town Center,City Center,2031,2040,1,,,,,"{""rings"": [[[415395.8512000004, 4546928.454], ..."
1,2,2,0.4,NaN,0.6,NaN,2019.0,0,1,1,1,1,0,0,0,01111000,WFRCCenters,WFRC,Layton Industrial Center,Industrial District,2031,2040,1,,,,,"{""rings"": [[[416374.0889999997, 4547760.0077],..."
2,3,3,30.0,100.0,2.0,4.0,2019.0,0,1,1,1,1,0,0,0,01111000,WFRCCenters,WFRC,Layton Midtown,Urban Center,2019,2030,1,,,,,"{""rings"": [[[417032.33100000024, 4549527.56279..."
3,4,4,30.0,100.0,1.9,4.0,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Layton Downtown,Urban Center,2031,2040,1,,,,,"{""rings"": [[[419554.49820000026, 4545424.31719..."
4,5,5,10.0,50.0,0.8,1.5,2037.0,1,1,0,1,1,0,0,0,11011000,WFRCCenters,WFRC,Gentile & Fairfield Town Center,City Center,2041,2050,1,,,,,"{""rings"": [[[420429.2611999996, 4546442.0261],..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,219,219,26.0,50.0,1.5,1.5,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Independence,City Center,2019,2030,1,,,,,"{""rings"": [[[420921.21190000046, 4479707.3002]..."
219,220,220,25.0,100.0,1.3,4.0,2037.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Porter Rockwell,Urban Center,2019,2030,1,,,,,"{""rings"": [[[423146.09520000033, 4480905.1357]..."
220,221,221,50.0,50.0,1.5,1.5,NaN,0,0,0,0,0,0,0,0,00000000,WFRCCenters,WFRC,Washington Terrace US 89,City Center,2031,2040,1,,,,,"{""rings"": [[[418307.74650000036, 4557001.59699..."
221,222,222,25.0,50.0,1.2,1.5,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Meadowbrook Station,City Center,2031,2040,1,,,,,"{""rings"": [[[424117.1978000002, 4503854.3684],..."


In [16]:
policy_sdf = non_phased.append(df)
policy_sdf.spatial.to_featureclass(location=os.path.join(gdb2, 'policy_override_polygons_2019_phased'),sanitize_columns=False)

In [17]:
parcels = r"E:\Tasks\Replace_HAFB_Parcels\Outputs\remm_base_year_20220513.gdb\parcels_2019"
parcels_pts = arcpy.FeatureToPoint_management(parcels, os.path.join(gdb, '_02_parcels_pts'), "INSIDE")

parcels_pts_lyr = arcpy.MakeFeatureLayer_management(parcels_pts, 'parcels_pts_lyr') 
arcpy.SelectLayerByAttribute_management(parcels_pts_lyr, 'NEW_SELECTION', "building_type_id in (6,9,10,13,14,16)")

arcpy.DeleteFeatures_management(parcels_pts_lyr)

arcpy.DeleteField_management(parcels_pts_lyr, 
                             ["CO_NAME", "TAZID", "COUNTY_ID","PARCEL_ID", "TOTAL_MKT_VALUE", "land_value", "residential_units", "building_sqft","FLOORS_CNT", "year_built",
                              "EFFBUILT_YR", "IS_OUG","max_dua","max_far","max_height", "max_height", "type1", "type2", "type3", "type4","type5", "type6", "type7", "type8",
                              "agriculture", "basebldg","NoBuild","redev_friction", "building_type_id","x", "y", "parcel_acres","Split","Split_Factor","parcel_id_REMM_old",
                              "residential_price","non_residential_price","non_residential_sqft"])

<Result 'parcels_pts_lyr'>

In [18]:
# parcels_pts_df = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, '_02_parcels_pts'))
# parcels_pts_df = parcels_pts_df[['parcel_id_REMM','SHAPE']].copy()
# parcels_pts_df.spatial.to_featureclass(location=os.path.join(gdb, '_03_parcels_pts'),sanitize_columns=False)

In [39]:
# use spatial join to summarize max dua
target_features = parcels_pts_lyr
join_features = os.path.join(gdb2, 'policy_override_polygons_2019_phased')
output_features = os.path.join(gdb, "_03_parcels_pts_policy_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# max_dua
fieldindex = fieldmappings.findFieldMapIndex('max_dua')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)


# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_MANY', "KEEP_COMMON", 
                           fieldmappings, "INTERSECT")

In [45]:
parcel_pts_policy_sdf = pd.DataFrame.spatial.from_featureclass(sj[0])
parcel_pts_policy_sdf

,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,parcel_id_REMM,building_type,TAZID_832,TAZID_900,ORIG_FID,id,max_dua,max_dua_new,max_far,max_far_new,year,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,locnote,mponote,AreaName,AreaType,phase_begin_year,phase_end_year,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,SHAPE
0,1,1,1,137,1,Multi Family Res,611,832.0,1,137.0,25.0,50.0,1.3,1.5,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Bountiful Downtown,City Center,2019.0,2030.0,1.0,,,,,"{""x"": 425848.5560999997, ""y"": 4526928.1381, ""s..."
1,2,1,3,134,3,Industrial,620,867.0,3,134.0,0.4,NaN,0.4,NaN,2019.0,0,0,1,0,0,0,0,0,00100000,WFRCCenters,WFRC,North Salt Lake / Woods Cross Industrial District,Industrial District,2019.0,2030.0,1.0,,,,,"{""x"": 421713.90490000043, ""y"": 4524888.319, ""s..."
2,3,1,5,138,5,Industrial,605,825.0,5,138.0,10.0,25.0,0.8,1.0,2019.0,1,1,0,1,1,0,0,0,11011000,WFRCCenters,WFRC,Gateway Crossing,Neighborhood Center,2031.0,2040.0,1.0,,,,,"{""x"": 424912.2796, ""y"": 4527549.851600001, ""sp..."
3,4,1,6,141,6,Industrial,576,791.0,6,141.0,0.0,NaN,0.5,NaN,2019.0,0,0,0,1,1,0,0,0,00011000,WFRCCenters,WFRC,Sheep Lane Employment District,Employment District,2031.0,2040.0,1.0,,,,,"{""x"": 424126.7297, ""y"": 4534739.7322, ""spatial..."
4,5,1,13,134,13,Industrial,643,881.0,13,134.0,0.4,NaN,0.4,NaN,2019.0,0,0,1,0,0,0,0,0,00100000,WFRCCenters,WFRC,North Salt Lake / Woods Cross Industrial District,Industrial District,2019.0,2030.0,1.0,,,,,"{""x"": 421729.23759999964, ""y"": 4522131.6081000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37244,37245,1,708671,208,762164,Agriculture,564,775.0,708671,209.0,30.0,100.0,1.9,4.0,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Station Park,Urban Center,2019.0,2030.0,1.0,,,,,"{""x"": 422909.9238999998, ""y"": 4537505.27820000..."
37245,37246,1,708674,31,762167,Agriculture,484,690.0,708674,31.0,0.0,NaN,0.4,NaN,2019.0,0,0,1,0,0,0,0,0,00100000,WFRCCenters,WFRC,East Gate Business Center,Employment District,2031.0,2040.0,1.0,,,,,"{""x"": 421103.24509999994, ""y"": 4551727.4039999..."
37246,37247,1,708675,31,762168,Empty Buildable,484,690.0,708675,31.0,0.0,NaN,0.4,NaN,2019.0,0,0,1,0,0,0,0,0,00100000,WFRCCenters,WFRC,East Gate Business Center,Employment District,2031.0,2040.0,1.0,,,,,"{""x"": 421103.24509999994, ""y"": 4551727.4039999..."
37247,37248,1,708676,31,762169,Agriculture,483,691.0,708676,31.0,0.0,NaN,0.4,NaN,2019.0,0,0,1,0,0,0,0,0,00100000,WFRCCenters,WFRC,East Gate Business Center,Employment District,2031.0,2040.0,1.0,,,,,"{""x"": 421106.2304999996, ""y"": 4551944.31640000..."


In [46]:
# if year is missing, use phase year
parcel_pts_policy_sdf.loc[(parcel_pts_policy_sdf['phase_begin_year'] > 0) & (parcel_pts_policy_sdf['year'].isna()== True), 'year'] = parcel_pts_policy_sdf['phase_begin_year']
parcel_pts_policy_sdf = parcel_pts_policy_sdf[(parcel_pts_policy_sdf['year'] > 0)]

In [47]:
parcel_pts_policy_sdf = parcel_pts_policy_sdf[['parcel_id_REMM', 'max_dua','max_far','year','type1','type2','type3','type4','type5','type6','type7','type8','locnote','mponote']].copy()

In [48]:
parcel_pts_policy_sdf.rename({'parcel_id_REMM': 'parcel_id'}, axis=1, inplace=True)
parcel_pts_policy_sdf.to_csv(r'Outputs\zoning_parcels_p.csv', index=False)

In [49]:
# import subprocess
os.startfile(r".\Policy_Override_Review.aprx")